In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

## Data quality verification

#### 3. Data requirements

The data requirements for this project combine both business and technical requirements. The business requirements are based on the domain knowledge of the project, while the technical requirements are based on the data types and formats required for the analysis. The data requirements will help ensure that the data is of high quality and suitable for further analysis and modeling.

EDA has shown that only some columns are important for future use, so we will define the requirements for these columns:
- We work only with data of 2022 year, so `FlightDate` should be in 2022 year.
- Time features should be given in the correct format of "hhmm", where hh is 00-23, mm is 00-59. Some of such time features are `CRSDepTime`, `CRSArrTime`.
- `OriginAirportID`, `DestAirportID`, `Operating_Airline`, `Tail_Number` should be in the correct format for airport and airline codes. IDs are numbers, OperatingAirline is a two-character string, Tail_Number is a string of digits and letters.
- `Cancelled` is a binary feature, so it should be False or True.
- Some features such as `Tail_Number` are not relevant for cancelled flights, so we should check that these features are present for non-cancelled flights, and are missing for cancelled flights.

We can define types for important columns in data in the following way:
- `FlightDate`: date, format "YYYY-MM-DD"
- `Operating_Airline`: string, length 2
- `OriginAirportID`, `DestAirportID`: integer
- `Cancelled`: boolean
- `CRSDepTime`, `CRSArrTime`: time, format "hhmm"
- `CRSElapsedTime`: integer, minutes
- `Tail_Number`: string (can be None for cancelled flights)
- `DepDelay`, `ActualElapsedTime`: integer, minutes (can be None for cancelled flights)
- `Distance`: integer, miles

For testing the quality of data, we will use Great Expectations library. It allows to define expectations for data, and then automatically check if the data meets these expectations. We will define expectations for the columns mentioned above, and then check if the data meets these expectations.

In [3]:
from src.data_quality import load_context_and_sample_data

# Create Great Expectations context and load data
context, da = load_context_and_sample_data("../services", "../data/samples/sample.csv")
batch_request = da.build_batch_request()

In [4]:
# Create expectation suite and retrieve validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="sample_validation",
)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-02-25,Alaska Airlines Inc.,YAK,JNU,False,False,1720,1701.0,0.0,-19.0,...,1713.0,1750.0,5.0,1811,-16.0,0.0,-2.0,1800-1859,1,0
1,2022-02-25,Alaska Airlines Inc.,SIT,JNU,False,False,1920,2031.0,71.0,71.0,...,2037.0,2100.0,5.0,2005,60.0,1.0,4.0,2000-2059,1,0
2,2022-02-25,Alaska Airlines Inc.,KTN,SIT,False,False,1734,1902.0,88.0,88.0,...,1919.0,1954.0,5.0,1830,89.0,1.0,5.0,1800-1859,1,0
3,2022-02-25,Alaska Airlines Inc.,SEA,KTN,False,False,1540,1555.0,15.0,15.0,...,1607.0,1649.0,5.0,1647,7.0,0.0,0.0,1600-1659,3,0
4,2022-02-25,Alaska Airlines Inc.,ANC,FAI,False,False,2335,12.0,37.0,37.0,...,24.0,101.0,3.0,34,30.0,1.0,2.0,0001-0559,2,0


Let's define our expectations for the sample data. We do it in notebook to execute them interactively.

In [5]:
# Expectations for "FlightDate" column
validator.expect_column_values_to_not_be_null(
    column="FlightDate", meta={"dimension": "Completeness"}
)
validator.expect_column_values_to_be_between(
    column="FlightDate",
    min_value="2022-01-01",
    max_value="2022-12-31",
    meta={"dimension": "Timelessness"},
)
validator.expect_column_values_to_match_regex(
    column="FlightDate", regex="^\d{4}-\d{2}-\d{2}$", meta={"dimension": "Validity"}
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 815664,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [6]:
# Expectations for "Operating_Airline" column
validator.expect_column_values_to_not_be_null(
    column="Operating_Airline", meta={"dimension": "Completeness"}
)
validator.expect_column_values_to_match_regex(
    column="Operating_Airline",
    regex="^[A-Z0-9]{2}$",
    meta={"dimension": "Validity"},
)
validator.expect_column_unique_value_count_to_be_between(
    column="Operating_Airline",
    min_value=1,
    max_value=100,
    meta={"dimension": "Uniqueness"},
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 21
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [7]:
# Expectations for "OriginAirportID" column
validator.expect_column_values_to_not_be_null(
    column="OriginAirportID", meta={"dimension": "Completeness"}
)
validator.expect_column_unique_value_count_to_be_between(
    column="OriginAirportID",
    min_value=1,
    max_value=1000000,
    meta={"dimension": "Uniqueness"},
)
validator.expect_column_unique_value_count_to_be_between(
    column="OriginAirportID",
    min_value=1,
    max_value=10000,
    meta={"dimension": "Uniqueness"},
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 372
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [8]:
# Expectations for "DestAirportID" column
validator.expect_column_values_to_not_be_null(
    column="DestAirportID", meta={"dimension": "Completeness"}
)
validator.expect_column_unique_value_count_to_be_between(
    column="DestAirportID",
    min_value=1,
    max_value=1000000,
    meta={"dimension": "Uniqueness"},
)
validator.expect_column_unique_value_count_to_be_between(
    column="DestAirportID",
    min_value=1,
    max_value=10000,
    meta={"dimension": "Uniqueness"},
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 372
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [9]:
# Expectations for "Cancelled" column
validator.expect_column_values_to_not_be_null(
    column="Cancelled", meta={"dimension": "Completeness"}
)
validator.expect_column_values_to_be_in_set(
    column="Cancelled", value_set=[False, True], meta={"dimension": "Validity"}
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 815664,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
# Expectations for "CRSDepTime" column
validator.expect_column_values_to_not_be_null(
    column="CRSDepTime", meta={"dimension": "Completeness"}
)
validator.expect_column_max_to_be_between(
    column="CRSDepTime", min_value=0, max_value=2400, meta={"dimension": "Accuracy"}
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2359
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [11]:
# Expectations for "CRSArrTime" column
validator.expect_column_values_to_not_be_null(
    column="CRSArrTime", meta={"dimension": "Completeness"}
)
validator.expect_column_max_to_be_between(
    column="CRSArrTime", min_value=0, max_value=2400, meta={"dimension": "Accuracy"}
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2359
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [12]:
# Expectations for "Tail_Number" column
validator.expect_column_values_to_match_regex(
    column="Tail_Number", regex="^[A-Z0-9]{5,6}$", meta={"dimension": "Validity"}
)
validator.expect_column_unique_value_count_to_be_between(
    column="Tail_Number",
    min_value=1,
    max_value=10000,
    meta={"dimension": "Uniqueness"},
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 5817
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
# Expectations for "CRSElapsedTime" column
validator.expect_column_values_to_not_be_null(
    column="CRSElapsedTime", meta={"dimension": "Completeness"}
)
validator.expect_column_max_to_be_between(
    column="CRSElapsedTime",
    min_value=0,
    max_value=8000,
    meta={"dimension": "Consistency"},
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 684.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
# Expectations for "DepDelay" column
validator.expect_column_max_to_be_between(
    column="DepDelay",
    min_value=-1000,
    max_value=10000,
    meta={"dimension": "Consistency"},
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2556.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [15]:
# Expectations for "ActualElapsedTime" column
validator.expect_column_max_to_be_between(
    column="ActualElapsedTime",
    min_value=0,
    max_value=8000,
    meta={"dimension": "Consistency"},
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 706.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
# Expectations for "Distance" column
validator.expect_column_values_to_not_be_null(
    column="Distance", meta={"dime      nsion": "Completeness"}
)
validator.expect_column_values_to_be_between(
    column="Distance",
    min_value=0,
    max_value=8000,
    meta={"dimension": "Consistency"},
)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 815664,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Let's now save the expectations suite and checkpoint.

In [17]:
# Save expectations and checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint = context.add_or_update_checkpoint(
    name="sample_checkpoint",
    validator=validator,
)

Now we can run the checkpoint to test that all expectations are valid for the current sample. This checkpoint will be run for all new samples automatically later.

In [18]:
checkpoint_result = checkpoint.run()

# Open report in browser
context.view_validation_result(checkpoint_result)

if checkpoint_result.success:
    print("Data quality verification passed successfully")
else:
    print("Data quality verification failed")

Calculating Metrics:   0%|          | 0/89 [00:00<?, ?it/s]

Data quality verification passed successfully
